# Imports

In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import itertools
import random
import pickle

# Check Gurobi license

In [2]:
try:

    # Create a new model
    m = gp.Model("2.17")

    # Create variables
    c = np.array([1, 2, 3])
    A = np.array([ [1, 2, 3], [1, 1, 0] ])
    b = np.array([4, 1])
    x_prev = np.array([1,2,3])
    x = m.addMVar(3, lb=0.0)
    m.setObjective(c @ x)
    m.addConstr(A@x == b)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % m.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-11
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x1990d0aa
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.000000000e+00
C0 1
C1 0
C2 1
Obj: 4


# Functions

In [4]:
def min_Q_(c: dict, W: dict, h: dict, T: dict, G: dict, g: dict, x_prev: dict, k: int, t: int) -> tuple:

    """
    This function minimizes the function `c[t][k] @ x_t + phi` subject to the following constraints:

    * `W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev`
    * `- G[t+1] @ x_t + phi >= g[t+1]`

    Parameters TODO: Change back to dicts
    ----------
    c : numpy.ndarray
        A 2D array of cost coefficients.
    W : numpy.ndarray
        A 2D array of transition matrices.
    h : numpy.ndarray
        A 1D array of holding costs.
    T : numpy.ndarray
        A 2D array of transfer matrices.
    G : numpy.ndarray
        A 2D array of constraint matrices.
    g : numpy.ndarray
        A 1D array of constraint bounds.
    x_prev : numpy.ndarray
        A 1D array of previous state values.
    k : int
        A scenario index.
    t : int
        A time index.

    Returns
    -------
    x_t_val : numpy.ndarray
        A 1D array of optimal state values.
    obj_val : float
        The optimal objective value.
    pi : float
        The dual value of the first constraint.
    rho : float
        The dual value of the second constraint.
    """

    m = gp.Model("Q_")
    
    x_t = m.addMVar(shape = len(c[t][k]))#, lb=0.0)
    phi = m.addVar()
    
    m.setObjective(c[t][k] @ x_t + phi)
    
    sequence_constraint = m.addConstr(W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev, name = "sequence_constraint")
    cut_constraint = m.addConstr(- G[t+1] @ x_t + phi >= g[t+1] , name = "cut_constraint") #*np.ones_like(g[t+1])
    #print(G[t+1])
    #print(g[t+1])
    m.update()
    #print(f'RHS: {cut_constraint.getAttr("RHS")}')
        
    # Solve
    m.setParam( 'OutputFlag', False )
    m.optimize()
    
    # Primal and objective value
    x_t_val = x_t.getAttr("x")
    obj_val = m.getAttr("ObjVal")
    
    # Duals
    pi = sequence_constraint.getAttr("Pi")
    rho = cut_constraint.getAttr("Pi")

    # Return results
    return x_t_val, obj_val, pi, rho


In [5]:
def initialize_cut_matrices(c: dict, phi_: dict, t_max: int) -> tuple[dict]:
    assert len(c) == t_max
    G = {}
    g = {}

    for t in range(2,t_max+1):
        G[t] = np.zeros(shape = (1, len(c[t-1][0])))
        g[t] = np.array([phi_[t]])

    G[t_max+1] = np.zeros(shape = (1, len(c[t_max][0])))
    g[t_max+1] = np.array([0])

    return G, g

In [6]:
def calculate_new_cut(q: dict, p: dict, h: dict[int, dict], T: dict[int, dict], 
                      pi: dict[int, dict], rho: dict[int, dict], g: dict, 
                      i: int, t: int, scenario: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculates the new cut, i.e. beta and alpha based on the input dicts and dual variables.
    
    Args:
    - q: A dictionary representing the number of elements for each time period.
    - p: A dictionary representing the probabilities of choosing an element for each time period.
    - h: A dictionary of dictionaries representing the values for each element at each time period.
    - T: A dictionary of dictionaries representing the transition matrix for each element at each time period.
    - pi: A dictionary of dictionaries of dictionaries representing the transition probabilities of each element at each time period.
    - rho: A dictionary of dictionaries of dictionaries representing the values of each constraint at each time period.
    - g: A dictionary representing the objective function values for each time period.
    
    Returns:
    A tuple containing the values of beta and alpha.
    """
    beta = - sum(p[t][j] * np.matmul(pi[i][t][scenario][j],T[t-1][j]) for j in range(q[t]))
    alpha = sum(p[t][j] * (np.matmul(pi[i][t][scenario][j],h[t][j]) + np.dot(rho[i][t][scenario][j],g[t+1])) for j in range(q[t]))
    return beta, alpha

In [7]:
def add_cut(G_t: np.array, g_t: np.array, beta: np.array, alpha: np.array) -> tuple[np.array, np.array]:
    assert G_t.shape[0] == g_t.shape[0]
    
    G_t = np.vstack((G_t,beta))
    g_t = np.hstack((g_t,alpha))

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)
    G_t = hstacked_unique[:, :-1]
    g_t = hstacked_unique[:, -1]
    return G_t, g_t


In [8]:
def remove_cut(G_t: np.ndarray, g_t: np.ndarray, beta: np.ndarray, alpha: np.ndarray) -> tuple[np.ndarray, np.ndarray]:

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)

    plane = np.hstack((beta, alpha))
    hstacked_removed = hstacked_unique[(hstacked_unique != plane).any(axis=1)]

    G_t = hstacked_removed[:, :-1]
    g_t = hstacked_removed[:, -1]
    return G_t, g_t

In [10]:
def calculate_v_upper(c: dict,x_i: dict, sampled_scenarios: list, S: dict, t_max: int) -> tuple[float]:

    """
    Calculates the upper bound extimate for the value v and the standard deviation of the estimator
    """
    
    v_i = {scenario: sum( np.dot(c[t][S[scenario][t]], x_i[t][scenario]) for t in range(1, t_max+1) ) for scenario in sampled_scenarios}
    v_upper_i = sum( v_i[scenario] for scenario in sampled_scenarios )
    v_upper_i = v_upper_i / len(sampled_scenarios)

    sigma_v_i = sum( np.square(v_i[scenario] - v_upper_i) for scenario in sampled_scenarios )
    sigma_v_i = sigma_v_i / ( (len(sampled_scenarios)-1) * len(sampled_scenarios) ) # Added to have standardisation
    sigma_v_i = np.sqrt(sigma_v_i)

    return v_upper_i, sigma_v_i


# SDDP

In [36]:
def SDDP(t_max: int, i_max: int, p: dict, q: dict, S: dict, c: dict, W: dict, h: dict, T: dict, phi_: np.array, no_of_samples: int, x_0: np.array) -> tuple:
    i = 0
    x = {}
    Q_ = {}
    pi = {}
    rho = {}
  
    v_lower = {}
    v_upper = {}
    sigma_v_upper = {}  

    # Initialize dicts
    for i in range(1, i_max+1):
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        # for scenario in list(S.keys()):
        #     x[i][0][scenario] = x_0
        
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
            # for scenario in list(S.keys()):
            #     pi[i][t][scenario] = {}
            #     rho[i][t][scenario] = {}

    print(f'dicts initialized, {len(list(S.keys()))} scenarios')
    
    
    # Initialize cut matrices
    G, g = initialize_cut_matrices(c=c, phi_=phi_, t_max=t_max)
    print('cut approximations initialized')
    
    
    i = 1

    while i <= i_max:
        
        ## FORWARD PASS
        print(f"Forward pass: {i}")
        sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
        for t in range(1,t_max+1):
            for scenario in sampled_scenarios:
                x[i][0][scenario] = x_0
                #print(f"t: {t}, scenario: {scenario}")
                k = S[scenario][t]
                x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t) # Seems good
        
        # Calculate upper bound estimate for v
        v_upper[i], sigma_v_upper[i] = calculate_v_upper(c, x[i], sampled_scenarios, S, t_max)
        print(f"v upper {i}: {v_upper[i]}, sigma v upper {i}: {sigma_v_upper[i]}")
    
        
        ## BACKWARD PASS
        for t in reversed(range(2,t_max+1)):
            for scenario in sampled_scenarios:

                # Initialize duals
                pi[i][t][scenario] = {}
                rho[i][t][scenario] = {}

                # Compute duals (pi, rho) for the scenario
                for j in range(q[t]):
                    x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t)
                
                if t != 1:
                    # Calculate new cut (beta, alpha)
                    beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)

                    # Add new cut (beta, alpha) to G, g:
                
                    G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                    
        # Finally solve stage t = 1 to get v_lower
        t = 1
        x[i][0][0] = x_0
        x[i][t][0], Q_[i][t][0], pi_temp, rho_temp = min_Q_(c,W,h,T,G,g,x[i][t-1][0],0,t)

        v_lower[i] = Q_[i][t][0]
        print(f"v lower {i}: {v_lower[i]}")
        i = i + 1
        
    return x, Q_, v_lower, v_upper

# BL-SDDP

In [50]:
def BL_SDDP(t_max: int, i_max: int, z_max: int, p: dict, q: dict, S: dict[int, tuple[int]], c: dict, W: dict, h: dict, T: dict, 
            phi_: np.array, no_of_samples: int, batch_size: int, x_0: np.array) -> tuple:
    
    assert batch_size <= no_of_samples, "Batch size larger than number of samples per stage"


    x = {}
    Q_ = {}
    pi = {}
    rho = {}
    
    v_lower = {}
    v_upper = {}
    sigma_v_upper = {}

    memory = {}
    
    # Initialize cut matrices
    G, g = initialize_cut_matrices(c=c, phi_=phi_, t_max=t_max)
    print('cut approximations initialized')


     # Initialize dicts
    for i in range(1, i_max+1):
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        # for scenario in list(S.keys()):
        #     x[i][0][scenario] = x_0
        
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
            # for scenario in list(S.keys()):
            #     pi[i][t][scenario] = {}
            #     rho[i][t][scenario] = {}
            

        for t in range(1,t_max+1):
            memory[t] = {}

    print(f'dicts initialized, {len(list(S.keys()))} scenarios')
    

    i = 1

    while i <= i_max:
        print(f'Iteration {i}:')

        z = 0
        while z < z_max:


            # Sample no_of_samples scenarios
            sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
            print(f"Sampled scenarios: {sampled_scenarios}")
            
            ## FORWARD PASS

            # Compute actions (forward)
            for t in range(1,t_max+1):
                for scenario in sampled_scenarios:
                    x[i][0][scenario] = x_0

                    k = S[scenario][t]
                    x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t)
        
                    # Add trial action to memory
                    memory[t][(i,scenario)] = {"action": x[i][t][scenario]}


            # Calculate upper bound estimate for v
            v_upper[i], sigma_v_upper[i] = calculate_v_upper(c, x[i], sampled_scenarios, S, t_max)
            print(f"v upper {i}: {v_upper[i]}, sigma v upper {i}: {sigma_v_upper[i]}")


            ## BACKWARD PASS
            
            for t in reversed(range(2,t_max+1)):
                for scenario in sampled_scenarios:
                    
                    # Initialize duals
                    pi[i][t][scenario] = {}
                    rho[i][t][scenario] = {}

                    # Compute duals (pi, rho) for the scenario
                    for j in range(q[t]):
                    
                        x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t)
                    
                    if t != 1:
                        # Calculate new cut (beta, alpha)
                        beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)
  
                        # Add new cut (beta, alpha) to G, g:
                    
                        G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                        # Store cut
                        memory[t][(i,scenario)]["beta"] = beta
                        memory[t][(i,scenario)]["alpha"] = alpha

            # Increase z
            z = z + no_of_samples
            
        # Select batch keys
        batch_keys = {}
        for t in range(2, t_max+1):

            # Select batch_size number of keys from the memory for stage t
            batch_keys[t] = random.sample(list(memory[t]), batch_size)

            # Remove the K cuts belonging to the K selected actions
            for batch_key in batch_keys[t]:
                beta = memory[t][batch_key]["beta"]
                alpha = memory[t][batch_key]["alpha"]
                if t != 1:
                    G[t], g[t] = remove_cut(G[t], g[t], beta, alpha)
        

        # Backward pass around the K trial actions
        for t in reversed(range(2,t_max+1)):
            for batch_key in batch_keys[t]:
                
                # Get scenario and iteration of the original
                i_batch, scenario = batch_key

                # Initialize x_0
                x[i][0][scenario] = x_0

                # Initialize duals
                pi[i][t][scenario] = {}
                rho[i][t][scenario] = {}

                # Compute duals (pi, rho) for the scenario
                for j in range(q[t]):
                    # print((t, batch_key, i, i_batch))
                    # print(x[i_batch][t-1][scenario])
                    x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i_batch][t-1][scenario],j,t)
                
                # Calculate new cut (beta, alpha)
                beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)

                # Add new cut (beta, alpha) to G, g:
                if t != 1:
                    G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                # Store new cut in memory (exactly where the removed cut was taken from)
                memory[t][batch_key]["beta"] = beta
                memory[t][batch_key]["alpha"] = alpha

        # Finally solve stage t = 1 to get v_lower
        t = 1
        x[i][0][0] = x_0
        x[i][t][0], Q_[i][t][0], pi_temp, rho_temp = min_Q_(c,W,h,T,G,g,x[i][t-1][0],0,t)

        v_lower[i] = Q_[i][t][0]
        print(f"v lower {i}: {v_lower[i]}")
            
        i = i + 1

    return x, Q_, v_lower, v_upper





# Examples

## Data functions

In [13]:
def generate_scenarios(q: dict, t_max: int) -> dict:
    """
    Generates the scenario set S based on the stage uncertainties q and t_max
    """
    assert len(q) == t_max
    
    list_q = [range(q[t]) for t in range(1,t_max+1)]
    scenarios_raw = [w for w in itertools.product(*list_q)]
    S = {i: {t: scenarios_raw[i][t-1] for t in range(1,len(scenarios_raw[i])+1)} for i in range(len(scenarios_raw))}
    return S

## Toy example

In [61]:
## INPUT
t_max = 3

# Uncertainty
q = {t: 3 for t in range(2,t_max+1)}
q[1] = 1
print(q)

p = {t: np.array([0.3333,0.3334,0.3333]) for t in range(2,t_max+1)}

S = generate_scenarios(q=q, t_max=t_max)
print(S)


# Initial condition
x_0 = np.array([ 0, 0 ])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}

for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}

c[1][0] = np.array([ 1, 0 ])
c[2][0] = np.array([ 1, 0 ])
c[2][1] = np.array([ 1, 0 ])
c[2][2] = np.array([ 1, 0 ])
c[3][0] = np.array([ 1, 1 ])
c[3][1] = np.array([ 1, 1 ])
c[3][2] = np.array([ 1, 1 ])

W[1][0] = np.array([ [1, 1] ])
W[2][0] = np.array([ [1, -1] ])
W[2][1] = np.array([ [1, -1] ])
W[2][2] = np.array([ [1, -1] ])
W[3][0] = np.array([ [1, -1] ])
W[3][1] = np.array([ [1, -1] ])
W[3][2] = np.array([ [1, -1] ])

h[1][0] = np.array([ 6 ])
h[2][0] = np.array([ 4 ])
h[2][1] = np.array([ 5 ])
h[2][2] = np.array([ 6 ])
h[3][0] = np.array([ 1 ])
h[3][1] = np.array([ 2 ])
h[3][2] = np.array([ 4 ])

T[0][0] = np.array([ [0, 0] ])
T[1][0] = np.array([ [1, 0] ])
T[1][1] = np.array([ [1, 0] ])
T[1][2] = np.array([ [1, 0] ])
T[2][0] = np.array([ [1, 0] ])
T[2][1] = np.array([ [1, 0] ])
T[2][2] = np.array([ [1, 0] ])


# Algorithm parameters
i_max = 40
z_max = 1
no_of_samples = 8
batch_size = 1
phi_list = [-100 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])
print(phi_)


{2: 3, 3: 3, 1: 1}
{0: {1: 0, 2: 0, 3: 0}, 1: {1: 0, 2: 0, 3: 1}, 2: {1: 0, 2: 0, 3: 2}, 3: {1: 0, 2: 1, 3: 0}, 4: {1: 0, 2: 1, 3: 1}, 5: {1: 0, 2: 1, 3: 2}, 6: {1: 0, 2: 2, 3: 0}, 7: {1: 0, 2: 2, 3: 1}, 8: {1: 0, 2: 2, 3: 2}}
[   0 -100 -100 -100]


In [62]:
x, Q_, v_lower, v_upper = SDDP(t_max=t_max, i_max=i_max,
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples,
                                  x_0 = x_0)

dicts initialized, 9 scenarios
cut approximations initialized
Forward pass: 1
v upper 1: 7.75, sigma v upper 1: 0.8183170883849714
v lower 1: 4.312410945116886
Forward pass: 2
v upper 2: 6.859308208837665, sigma v upper 2: 0.4580234257406953
v lower 2: 5.5554444499999995
Forward pass: 3
v upper 3: 6.196420918651565, sigma v upper 3: 0.477585155608183
v lower 3: 6.166566661250375
Forward pass: 4
v upper 4: 6.374999994375562, sigma v upper 4: 0.4406772387726661
v lower 4: 6.222144439999999
Forward pass: 5
v upper 5: 6.125, sigma v upper 5: 0.479490056503484
v lower 5: 6.222144439999999
Forward pass: 6
v upper 6: 6.5, sigma v upper 6: 0.3779644730092272
v lower 6: 6.222144439999999
Forward pass: 7
v upper 7: 6.0, sigma v upper 7: 0.4225771273642583
v lower 7: 6.222144439999999
Forward pass: 8
v upper 8: 6.5, sigma v upper 8: 0.3779644730092272
v lower 8: 6.222144439999999
Forward pass: 9
v upper 9: 6.25, sigma v upper 9: 0.49099025303098287
v lower 9: 6.222144439999999
Forward pass: 10
v 

In [64]:
x[40][1][0]

array([3., 3.])

In [65]:
x, Q_, v_lower, v_upper = BL_SDDP(t_max=t_max, i_max=i_max, z_max=z_max, 
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples, batch_size = batch_size,
                                  x_0 = x_0)

cut approximations initialized
dicts initialized, 9 scenarios
Iteration 1:
Sampled scenarios: [3, 5, 1, 7, 2, 6, 0, 8]
v upper 1: 7.625, sigma v upper 1: 0.8223984956906754
v lower 1: 4.312410945116886
Iteration 2:
Sampled scenarios: [8, 7, 2, 3, 4, 0, 5, 1]
v upper 2: 6.812410945116887, sigma v upper 2: 0.4629100498862757
v lower 2: 5.5554444499999995
Iteration 3:
Sampled scenarios: [0, 8, 2, 4, 5, 3, 6, 7]
v upper 3: 6.446420918651565, sigma v upper 3: 0.4297940984391966
v lower 3: 6.166566661250375
Iteration 4:
Sampled scenarios: [6, 5, 3, 1, 2, 7, 0, 8]
v upper 4: 6.499999997187782, sigma v upper 4: 0.4225771273642583
v lower 4: 6.222144439999999
Iteration 5:
Sampled scenarios: [6, 3, 0, 2, 1, 8, 4, 7]
v upper 5: 6.125, sigma v upper 5: 0.479490056503484
v lower 5: 6.222144439999999
Iteration 6:
Sampled scenarios: [1, 5, 0, 3, 4, 2, 7, 8]
v upper 6: 6.0, sigma v upper 6: 0.4225771273642583
v lower 6: 6.222144439999999
Iteration 7:
Sampled scenarios: [2, 1, 4, 5, 8, 6, 7, 0]
v upper

In [67]:
x[40][1][0]

array([3., 3.])

## Energy example - single household

In [68]:
## INPUT

t_max = 24

# Uncertainty
q = {t: 2 for t in range(2,t_max+1)}
q[1] = 1

p = {t: np.array([0.45,0.55]) for t in range(2,t_max+1)}

S = generate_scenarios(q=q, t_max=t_max)
print(f"No. of scenarios: {len(S)}")

# Parameters
from_grid_max = 25
to_grid_max = 25
soc_max = 100

price_from_grid = {t: {} for t in range(1,t_max+1)}
price_to_grid = {t: {} for t in range(1,t_max+1)}
gen_less_demand = {t: {} for t in range(1,t_max+1)}

for t in range(1, 2):
    price_from_grid[t] = {0: 0.3}
    price_to_grid[t] = {0: 0.15}
    gen_less_demand[t] = {0: 0}

for t in range(2, 7):
    price_from_grid[t] = {0: 0.2, 1: 0.4}
    price_to_grid[t] = {0: 0.1, 1: 0.2} 
    gen_less_demand[t] = {0: 0, 1: 10}

for t in range(7, t_max+1):
    price_from_grid[t] = {0: 0.4, 1: 0.6}
    price_to_grid[t] = {0: 0.2, 1: 0.4}
    gen_less_demand[t] = {0: -20, 1: -5}

# Initial condition
x_0 = np.array([0,0,10,0,0,0])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}
for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}
    for j in range(q[t]):
        c[t][j] = np.array([price_from_grid[t][j], -price_to_grid[t][j], 0, 0, 0, 0])
        W[t][j] = np.array([ [-1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1] ])
        h[t][j] = np.array([gen_less_demand[t][j], from_grid_max, to_grid_max, soc_max])
        T[t-1][j] = np.array([ [0, 0, -1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0] ])


No. of scenarios: 8388608


In [69]:
# Algorithm parameters
i_max = 10
z_max = 10
no_of_samples = 150
batch_size = 30
phi_list = [-20 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])

In [70]:
result_sddp = {}
result_sddp["x"], result_sddp["Q_"], result_sddp["v_lower"], result_sddp["v_upper"] = SDDP(t_max=t_max, i_max=i_max,
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples,
                                  x_0 = x_0)

dicts initialized, 8388608 scenarios
cut approximations initialized
Forward pass: 1
v upper 1: 92.84, sigma v upper 1: 0.9719018924719734
v lower 1: 20.96820282092891
Forward pass: 2
v upper 2: 69.94424572263723, sigma v upper 2: 1.050999316478937
v lower 2: 48.52743750000006
Forward pass: 3
v upper 3: 74.46626759834373, sigma v upper 3: 1.1407425150588109
v lower 3: 55.85000000000009
Forward pass: 4
v upper 4: 68.48888346478535, sigma v upper 4: 1.029119153353896
v lower 4: 58.207466181061456
Forward pass: 5
v upper 5: 65.38250676378777, sigma v upper 5: 0.9574021036947636
v lower 5: 59.00374593750007
Forward pass: 6
v upper 6: 65.66176190476189, sigma v upper 6: 1.0717725838194636
v lower 6: 59.019637812500065
Forward pass: 7
v upper 7: 62.85333333333333, sigma v upper 7: 1.0301663238211933
v lower 7: 59.019637812500065
Forward pass: 8
v upper 8: 63.66, sigma v upper 8: 1.096458810871325
v lower 8: 59.019637812500065
Forward pass: 9
v upper 9: 65.23333333333333, sigma v upper 9: 1.01

In [71]:
with open("result_sddp_2.pkl", 'wb') as handle:
    pickle.dump(result_sddp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [60]:
result_sddp["x"][5][1][0]

array([ 0.,  0., 10., 25., 25., 90.])

In [72]:
result_blsddp = {}
result_blsddp["x"], result_blsddp["Q_"], result_blsddp["v_lower"], result_blsddp["v_upper"] = BL_SDDP(t_max=t_max, i_max=i_max, z_max=z_max, 
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples, batch_size = batch_size,
                                  x_0 = x_0)

cut approximations initialized
dicts initialized, 8388608 scenarios
Iteration 1:
Sampled scenarios: [5431410, 3111525, 664391, 2045105, 8012664, 6463937, 5802756, 4218993, 3398685, 1774472, 1692074, 1496843, 6862547, 4046830, 2099609, 7427002, 6149795, 8072954, 5361777, 3976332, 1408338, 936928, 7994747, 7644477, 2185025, 5258883, 4794911, 8039122, 1738120, 194831, 6233811, 2466853, 100185, 3558684, 4961655, 4042148, 5829830, 6641065, 956710, 5882118, 4274303, 1779957, 2373811, 6957790, 6832694, 3520077, 2827526, 5081867, 1025325, 334717, 5576800, 7017095, 3007259, 7734401, 640820, 5721830, 4889539, 807868, 6611366, 4034892, 4604802, 2594247, 7038449, 942962, 4338909, 142764, 4422877, 1166985, 2107786, 6892882, 4834475, 4967496, 6405864, 4157998, 147265, 6338014, 5175195, 7455314, 2391075, 6743552, 3835246, 8339525, 2426516, 7717220, 8043449, 1911232, 6849786, 4140750, 6915683, 2070095, 2034010, 5972390, 4822083, 8351983, 3880578, 2946099, 6809073, 6582988, 4922607, 2377827, 1543917, 7

In [73]:
result_blsddp["x"][5][1][0]

array([ 9.99963346,  0.        , 19.99963346, 15.00036654, 25.        ,
       80.00036654])

In [75]:
with open("result_blsddp_2.pkl", 'wb') as handle:
    pickle.dump(result_blsddp, handle, protocol=pickle.HIGHEST_PROTOCOL)